In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

In [ ]:
# --- 1. Define SRM Filter Kernels ---
# These are simplified versions of the 30+ filters found in the original SRM paper.
# They detect residuals in different directions and orders.

def get_srm_kernels():
    kernels = []
    
    # 1st order: Standard differences
    k1 = np.array([[0, 0, 0], [0, -1, 1], [0, 0, 0]], dtype=np.float32) # Horizontal
    k2 = np.array([[0, 0, 0], [0, -1, 0], [0, 1, 0]], dtype=np.float32) # Vertical
    
    # 2nd order: Edge detection (Laplacian-like)
    k3 = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]], dtype=np.float32)
    
    # 3rd order: Complex residuals (approximate specific SRM types)
    k4 = np.array([[-1, 2, -1], [2, -4, 2], [-1, 2, -1]], dtype=np.float32)
    
    # 5x5 High pass (captures wider dependencies)
    k5 = np.array([[-1, 2, -2, 2, -1], 
                   [2, -6, 8, -6, 2], 
                   [-2, 8, -12, 8, -2], 
                   [2, -6, 8, -6, 2], 
                   [-1, 2, -2, 2, -1]], dtype=np.float32) / 12.0

    kernels.extend([k1, k2, k3, k4, k5])
    return kernels

# --- 2. Feature Extractor ---

def extract_srm_features(image_path, kernels):
    """
    Reads an image, applies SRM filters, and returns statistical moments of the residuals.
    """
    try:
        # Read as grayscale
        img = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
        if img is None:
            return None
        
        # Determine crop (center crop is usually best to avoid border artifacts)
        h, w = img.shape
        crop_size = min(512, h, w) 
        cy, cx = h // 2, w // 2
        img_crop = img[cy - crop_size//2 : cy + crop_size//2, 
                       cx - crop_size//2 : cx + crop_size//2]
        
        img_float = img_crop.astype(np.float32)
        features = []
        
        for k in kernels:
            # Apply the filter
            residual = cv2.filter2D(img_float, -1, k)
            
            # To handle dynamic range, we look at the statistics of the residual
            # 1. Variance (Energy of the noise)
            # 2. Skewness (Asymmetry of noise distribution)
            # 3. Kurtosis (Tail heaviness - vital for detecting AI/editing)
            
            res_flat = residual.flatten()
            features.append(np.var(res_flat))
            features.append(skew(res_flat))
            features.append(kurtosis(res_flat))
            
        return np.array(features)
        
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None



In [ ]:
# --- 3. Main Execution ---

# Configuration
IMAGE_DIR = "/path/to/your/images" # <--- UPDATE THIS
ALLOWED_EXTS = {'.jpg', '.jpeg', '.png', '.tif', '.tiff', '.webp'}

# Load Kernels
kernels = get_srm_kernels()
print(f"Loaded {len(kernels)} SRM kernels.")

# Processing Loop
data_features = []
filenames = []

print("Starting feature extraction...")
path_obj = Path(IMAGE_DIR)
all_files = [p for p in path_obj.iterdir() if p.suffix.lower() in ALLOWED_EXTS]

for i, img_path in enumerate(all_files):
    if i % 10 == 0: print(f"Processing {i}/{len(all_files)}...")
    
    feats = extract_srm_features(img_path, kernels)
    if feats is not None:
        data_features.append(feats)
        filenames.append(img_path.name)

# Create DataFrame
X = np.array(data_features)
print(f"Extraction complete. Feature matrix shape: {X.shape}")

# --- 4. Clustering & Visualization ---

# A. Normalization (CRITICAL for variance/kurtosis data)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# B. Dimensionality Reduction (PCA -> t-SNE)
# We use PCA first to suppress noise before t-SNE
pca = PCA(n_components=min(10, X.shape[1]))
X_pca = pca.fit_transform(X_scaled)

tsne = TSNE(n_components=2, perplexity=min(30, len(X)-1), random_state=42)
X_embedded = tsne.fit_transform(X_pca)

# C. Clustering (K-Means)
# You might want to adjust n_clusters based on how many sources/cameras you expect
n_clusters = 3 
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(X_scaled)

# D. Plotting
df_viz = pd.DataFrame({
    'x': X_embedded[:, 0], 
    'y': X_embedded[:, 1], 
    'Cluster': labels, 
    'Filename': filenames
})

plt.figure(figsize=(12, 8))
sns.scatterplot(
    data=df_viz, x='x', y='y', hue='Cluster', 
    palette='viridis', s=100, alpha=0.8, edgecolor='k'
)

plt.title('SRM Noise Analysis Clusters (t-SNE)', fontsize=16)
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.legend(title='Cluster ID', bbox_to_anchor=(1.05, 1), loc='upper left')

# Optional: Annotate points to spot check
# for i in range(df_viz.shape[0]):
#    plt.text(df_viz.x[i]+0.2, df_viz.y[i]+0.2, df_viz.Filename[i], fontsize=8)

plt.tight_layout()
plt.show()